In [11]:
# IMPORTS
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

from scipy import stats

import geopandas as gpd
import numpy as np
from shapely.geometry import box

from typing import Tuple, Dict, Any, List


import panel as pn
from matplotlib.figure import Figure


from tkinter import Tk
from tkinter.filedialog import askopenfilenames

import cupy as cp  # CuPy for GPU acceleration



In [43]:
#FUNCTION DEFINITION - CPU

def geojson_to_numpy_grid_3d_batch(
    grid_size: Tuple[int, int],  # Grid size for the output array
    target_crs: str = "EPSG:3857"  # Web Mercator projection
) -> Tuple[np.ndarray, Dict[str, np.ndarray], Dict[str, Dict[Any, int]], List[Dict[str, Any]]]:
    # Open a file selection dialog for the user to select multiple GeoJSON files
    root = Tk()
    root.withdraw()  # Hide the root window
    root.attributes("-topmost", True)  # Bring the dialog to the front

    # Open the file selection dialog
    geojson_files = askopenfilenames(
        title="Select GeoJSON Files",
        filetypes=[("GeoJSON files", "*.geojson"), ("All files", "*.*")]
    )

    all_feature_grids = {}
    all_feature_mappings = {}
    geospatial_info_list = []

    for geojson_file in geojson_files:
        # Read the GeoJSON file
        gdf = gpd.read_file(geojson_file)

        # Reproject to target CRS
        gdf = gdf.to_crs(target_crs)

        # Get the total bounds of all geometries
        minx, miny, maxx, maxy = gdf.total_bounds

        # Create a fixed-size grid
        x = np.linspace(minx, maxx, grid_size[1] + 1)
        y = np.linspace(miny, maxy, grid_size[0] + 1)

        # Automatically extract all relevant feature columns, excluding geometry columns
        feature_columns = [col for col in gdf.columns if col != gdf.geometry.name]

        # Get the filename without extension for prefixing
        filename_prefix = os.path.splitext(os.path.basename(geojson_file))[0]

        # Dictionary to hold grids and category mappings for each feature column
        feature_grids = {}
        feature_mappings = {}

        # Store geospatial information for each file
        geospatial_info = {
            'transform': (minx, miny, maxx, maxy),
            'crs': target_crs,
            'file_name': filename_prefix
        }
        geospatial_info_list.append(geospatial_info)

        # Iterate over each feature column
        for feature_column in feature_columns:
            # Get unique categories and create a mapping to integers
            unique_categories = gdf[feature_column].unique()
            
            # Prefix each feature class with the filename
            category_to_int = {f"{filename_prefix}_{cat}": i for i, cat in enumerate(unique_categories)}

            # Initialize the 2D NumPy array with -1 (representing no data)
            grid = np.full(grid_size, -1, dtype=int)

            # Create a spatial index for faster intersection checks
            sindex = gdf.sindex

            # Pre-compute cell geometries
            cells = [box(x[j], y[i], x[j + 1], y[i + 1])
                     for i in range(grid_size[0])
                     for j in range(grid_size[1])]

            # Vectorized operations for intersection
            def process_cell(cell, possible_matches):
                if possible_matches.empty:
                    return -1
                intersections = possible_matches.geometry.intersection(cell)
                intersection_areas = intersections.area
                largest_intersection_idx = intersection_areas.idxmax()
                category = possible_matches.loc[largest_intersection_idx, feature_column]
                return category_to_int[f"{filename_prefix}_{category}"]

            # Iterate through each cell in the grid
            for idx, cell in enumerate(cells):
                i, j = divmod(idx, grid_size[1])

                # Use the spatial index to find potential intersecting polygons
                possible_matches_index = list(sindex.intersection(cell.bounds))
                if not possible_matches_index:
                    continue

                # Check for actual intersection and assign the feature value
                possible_matches = gdf.iloc[possible_matches_index]
                grid[i, j] = process_cell(cell, possible_matches)

            # Store the grid and category mapping for this feature
            feature_grids[f"{filename_prefix}_{feature_column}"] = grid
            feature_mappings[f"{filename_prefix}_{feature_column}"] = category_to_int

        # Merge with all feature grids and mappings
        all_feature_grids.update(feature_grids)
        all_feature_mappings.update(feature_mappings)

    # Stack all grids into a 3D array
    grid_3d = np.stack(list(all_feature_grids.values()), axis=0)

    return grid_3d, all_feature_grids, all_feature_mappings, geospatial_info_list


In [ ]:
#RUN FUNCTION

grid_size = (10, 10)  # Define the grid size

# Call the function
grid_3d, feature_grids, feature_mappings, geospatial_info_list = geojson_to_numpy_grid_3d_batch(grid_size)

# Print results
print("Shape of the 3D grid array:", grid_3d.shape)
print("Feature grids:", feature_grids.keys())
print("Feature mappings:", feature_mappings)
print("Geospatial information for each file:", geospatial_info_list)


In [ ]:
print(grid_3d[3, :5, :5])

In [ ]:
#PLOT

# Initialize the Panel extension
pn.extension()

# Function to plot a specific layer using Matplotlib
def plot_layer_bokeh(layer_index):
    fig = Figure(figsize=(4, 3))
    ax = fig.add_subplot(111)
    im = ax.imshow(grid_3d[layer_index], cmap='tab20', interpolation='nearest', aspect='auto')
    ax.set_title(f"Layer {layer_index + 1}: {list(feature_grids.keys())[layer_index]}")
    fig.colorbar(im, ax=ax, label='Classes')
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')
    return pn.pane.Matplotlib(fig, tight=True)

# Create a Panel widget for selecting the layer
layer_slider = pn.widgets.IntSlider(name='Layer Index', start=0, end=grid_3d.shape[0] - 1, step=1, value=0)

# Bind the plotting function to the slider value
panel = pn.bind(plot_layer_bokeh, layer_index=layer_slider)

# Display the Panel with the slider and plot
pn.Column(layer_slider, panel).servable()



In [ ]:
import panel as pn
import matplotlib.pyplot as plt
from matplotlib.figure import Figure

# Initialize the Panel extension
pn.extension()

# Function to plot a specific layer using Matplotlib
def plot_layer_bokeh(layer_index):
    # Debugging: Print information about the current layer being plotted
    print(f"Plotting Layer {layer_index + 1}/{grid_3d.shape[0]}: {list(feature_grids.keys())[layer_index]}")
    print(f"Min value in layer: {np.min(grid_3d[layer_index])}, Max value in layer: {np.max(grid_3d[layer_index])}")

    # Create the plot
    fig = Figure(figsize=(3, 4))
    ax = fig.add_subplot(111)
    im = ax.imshow(grid_3d[layer_index], cmap='tab20', interpolation='nearest', aspect='auto')
    ax.set_title(f"Layer {layer_index + 1}: {list(feature_grids.keys())[layer_index]}")
    fig.colorbar(im, ax=ax, label='Classes')
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')

    # Debugging: Display the array data for the current layer
    print("Layer data:\n", grid_3d[layer_index])

    return pn.pane.Matplotlib(fig, tight=True)

# Create a Panel widget for selecting the layer
layer_slider = pn.widgets.IntSlider(name='Layer Index', start=0, end=grid_3d.shape[0] - 1, step=1, value=0)

# Bind the plotting function to the slider value
panel = pn.bind(plot_layer_bokeh, layer_index=layer_slider)

# Display the Panel with the slider and plot
pn.Column(layer_slider, panel).servable()
